# Kernel Memory plugin for Semantic Kernel

This notebook shows the basic usage of Kernel Memory as a Semantic Kernel Plugin.

First of all, install the Kernel Memory SK Plugin and SK dependencies.

In [9]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.11.231121.1-preview"

using Microsoft.SemanticKernel;
using Microsoft.KernelMemory;

Installed Packages Microsoft.KernelMemory.SemanticKernelPlugin, 0.11.231121.1-preview Microsoft.SemanticKernel, 1.0.0-beta8

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with your OpenAI API Key, and your Memory Service API Key (if you set one).

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> MEMORY_API_KEY=<your KM web service API key>
> ```

In [ ]:
#r "nuget: dotenv.net, 3.1.2"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Let's setup Semantic Kernel as usual:

In [11]:
var kernel = new KernelBuilder()
    .WithOpenAIChatCompletionService(
        modelId: "gpt-3.5-turbo",
        apiKey: env["OPENAI_API_KEY"])
    .Build();

Here we define a simple semantic function, which depends on `{{memory.ask}}` to
fetch information from memory.

In [12]:
 var skPrompt = """
                Question to Kernel Memory: {{$input}}

                Kernel Memory Answer: {{memory.ask $input}}

                If the answer is empty say 'I don't know' otherwise reply with a preview of the answer, truncated to 15 words. Prefix with one emoji relevant to the content.
                """;

var myOracleFunction = kernel.CreateSemanticFunction(skPrompt);

Let's load Kernel Memory plugin into SK. 

Remember to start the memory service on localhost, otherwise change the URL and
point it to your Kernel Memory service endpoint. In that case you should also
provide the API key protecting your KM deployment.

In [13]:
var memoryServiceEndpoint = "http://127.0.0.1:9001/";
var memoryServiceAPIKey = env["MEMORY_API_KEY"];

var memoryConnector = new MemoryWebClient(memoryServiceEndpoint, memoryServiceAPIKey);

var pluginName = "memory";

var memoryPlugin = kernel.ImportFunctions(new MemoryPlugin(memoryConnector, waitForIngestionToComplete: true), pluginName);

Let's save some data:

1. storing some text
2. and importing a PDF document. 

To import data you can use either the web client connector or the plugin, here we use both just for demo purposes.

When working with SK Planners and Semantic Functions (like the function above) instead, you should always
use KM Plugin. Of course, you can also use the web client connector to create new plugins and customize
how you interact with your memories.

In [14]:
// Save a string using the web client
await memoryConnector.ImportTextAsync("Orion is a prominent set of stars visible during winter in " +
                                      "the northern celestial hemisphere. It is one of the 88 modern constellations; " +
                                      "it was among the 48 constellations listed by the 2nd-century astronomer Ptolemy. " +
                                      "It is named for a hunter in Greek mythology.", 
                                      documentId: "OrionDefinition");

// Save a PDF file using the plugin
var context = kernel.CreateNewContext();
context.Variables[MemoryPlugin.FilePathParam] = "NASA-news.pdf";
context.Variables[MemoryPlugin.DocumentIdParam] = "NASA001";
await memoryPlugin["SaveFile"].InvokeAsync(context);

Console.WriteLine("Memory updated.");

Memory updated.


Let's now interact with our memories using `myOracleFunction` function, asking questions, leveraging KM and LLMs to generate answers.

Our function internally uses KM Plugin to retrieve an answer, and then custom some semantic logic to decide how to format the output.

In [15]:
var answer = await myOracleFunction.InvokeAsync("any news from NASA about Orion?", kernel);
Console.WriteLine("Answer: " + answer);

Answer: 🚀 NASA has invited media to see the new test version of Orion spacecraft and recovery hardware.


In [16]:
var answer = await myOracleFunction.InvokeAsync("what is Orion?", kernel);
Console.WriteLine("Answer: " + answer);

Answer: 🌌 Orion is a constellation named after a hunter in Greek mythology and also a NASA spacecraft.
